# Классификация клиентов телеком компании

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Изучение файла

In [3]:
#открываем файл и печатаем первые 5 строк
import pandas as pd
#< импортируйте функцию train_test_split из библиотеки sklearn >
from sklearn.model_selection import train_test_split

#импортируем структуры данных для будущих моделей исслелования: "дерево решений", "случайный лес", "логистиечская регрессия"
#и метрику качества accuracy
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np 
from sklearn.dummy import DummyClassifier

df = pd.read_csv('/datasets/users_behavior.csv')
print(df.head())
df.info()
print(df.pivot_table(index=['is_ultra'], aggfunc ='count'))

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
          calls  mb_used  messages  minutes
is_ultra                                   
0          2229     2229      2229     2229
1           985      985       985      985


In [ ]:
#изменим тип данных в колонке  на bool
df['is_ultra'] = df['is_ultra'].astype(bool)
df.info()

## Разделение данных на выборки

In [4]:
#извлекаем признаки 
features = df.drop(['is_ultra'], axis=1)

#извлекаем целевой признак
target = df['is_ultra']



#< разделите данные на обучающую, валидационную и тестовую выборки >

df_train, df_train2 = train_test_split(df, test_size=0.40, random_state=12345, stratify=df['is_ultra'])
df_valid, df_test = train_test_split(df_train2, test_size=0.5, random_state=12345, stratify=df_train2['is_ultra'])

#print(df_test1)
# < создаем переменные для признаков и целевого признака >
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

print('Размер обучающей выборки:', df_train.shape)
print('Размер валидационной выборки:',df_valid.shape)
print('Размер тестовой выборки:',df_test.shape)
print('')
print('Распределение таргетов в датасете  df_train', df_train.pivot_table(index=['is_ultra'], aggfunc ='count'))

print('Распределение таргетов в датасете  df_test', df_test.pivot_table(index=['is_ultra'], aggfunc ='count'))

print('Распределение таргетов в датасете  df_valid', df_valid.pivot_table(index=['is_ultra'], aggfunc ='count'))

Размер обучающей выборки: (1928, 5)
Размер валидационной выборки: (643, 5)
Размер тестовой выборки: (643, 5)

Распределение таргетов в датасете  df_train           calls  mb_used  messages  minutes
is_ultra                                   
0          1337     1337      1337     1337
1           591      591       591      591
Распределение таргетов в датасете  df_test           calls  mb_used  messages  minutes
is_ultra                                   
0           446      446       446      446
1           197      197       197      197
Распределение таргетов в датасете  df_valid           calls  mb_used  messages  minutes
is_ultra                                   
0           446      446       446      446
1           197      197       197      197


## Исследование моделей

***Проверка наилучшей модел на алгоритме классификации — дерево решений***

In [6]:
#проверим модель, указывая глубину обученного дерева от 1 до 10
max_depth = None
best_accuracy = 0
best_min_samples = 0
for depth in range(1, 7):
    for samples in range (2,5):
        model = DecisionTreeClassifier(max_depth=depth, criterion='gini', random_state=12345,  min_samples_split=samples)
        model.fit(features_train, target_train) # # обучаем модель на тренировочной выборке
        predictions_valid = model.predict(features_valid) # посчитаем качество модели на валидационной выборке
        accuracy=accuracy_score(target_valid, predictions_valid)
        if accuracy > best_accuracy:
            max_depth = depth # сохраняем наилучшую глубину дерева
            best_accuracy = accuracy
            best_min_samples = samples
print('Наибольшая доля правильных ответов:', best_accuracy) 
print('Наилучшая глубина обученного дерева', max_depth)
print('Наилучшая min кол-во  объектов обучающей выборки в узле', best_min_samples)

Наибольшая доля правильных ответов: 0.7853810264385692
Наилучшая глубина обученного дерева 5
Наилучшая min кол-во  объектов обучающей выборки в узле 2


***Проверка наилучшей модел на алгоритме классификации — случайный лес***

In [8]:
#Обучим модели случайного леса с числом деревьев от 1 до 20
best_accuracy = 0
best_est = 0
for est in range(1, 40):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) #запишем модельв переменную, указав гиперпараметры, кол-во деревье
    model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
    predictions_valid = model.predict(features_valid)
    accuracy=accuracy_score(target_valid, predictions_valid) # посчитайте качество модели на валидационной выборке
    if accuracy > best_accuracy:
        best_accuracy = accuracy #наибольшая доля правильных ответов
        best_est = est  #сохраняем наилучшую глубину дерева 

print('Наибольшая доля правильных ответов:', best_accuracy) 
print('Наилучее число деревьев в модели', best_est)

Наибольшая доля правильных ответов: 0.7993779160186625
Наилучее число деревьев в модели 21


***Проверка наилучшей модел на алгоритме классификации - логистическая регрессия***

In [9]:
#обучим модель линейной регресии на количестве итераций =2000
model = LogisticRegression(random_state=1234, max_iter=2000)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
accuracy=accuracy_score(target_valid, predictions_valid)

print('Доля правильных ответов:', accuracy) 


Доля правильных ответов: 0.7387247278382582


***ВЫВОДЫ по точности построения моделей на основе обучающей выборки***

По результатам построения  3 моделей, способных проанализировать поведение клиентов и предложить им один из новых тарифов, можно сделать следующие выводы:
1. Наиболее точно и качествено, а сооветственро поможет принести компании принести большую прибыль работате модель "случайного леса", ее точность на обучающей выборке достигает 0.7993779160186625 при числе деревьев =21.
2. Затем следует модель "дерево решений" с точность 0.7853810264385692, глубиной обученого дерева = 5 и  min кол-во  объектов обучающей выборки в узле 2.
3. И на последнем месте стоит модель "логистической регресии" с точностью 0.7387247278382582. 

## Проверка модели на тестовой выборке

Проверять на тестовой выборке будем самую точную полученную модель - случаный лес, с параметрами: глубина обучения. = 21.

***Проверка модели случайный лес***

In [10]:
#наибольшая доля правильных ответов была получена при количестве деревье = 21

model = RandomForestClassifier(random_state=12345, n_estimators=21) #запишем модельв переменную, указав гиперпараметры, кол-во деревье
model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
predictions_test = model.predict(features_test) # посчитаем качество модели на тестовой выборке
accuracy=accuracy_score(target_test, predictions_test)

print('Доля правильных ответов на тестовой выборке:', accuracy)

Доля правильных ответов на тестовой выборке: 0.80248833592535


***ВЫВОДЫ ПО РЕЗУЛЬТАТАМ ИССЛЕДОВАНИЯ***


В нашу задачу входило построить модель, спобную проанализировать поведение клиентов мобильного оператора, которые уже выбрали один из 2 новых тарифов оператора, и  рекомендовать наиболее подходящий тариф новым клиентам.

В анализа были получены данные в объеме 3214 строк, их предобработка уже была проделана.

В работе было проделано следующее:
 - данные разбиты на обучающую, валидационную и тестовую выборку, с учетом сбалансированности таргетов в каждой. 
 - 3 модели "дерево решений", "случайный лес" и "логистическая регрессия" были обучены на обучающей выборке;
 - затем протестированы на валидационной  и по результате подбора гиперпараметров с помощью циклов, выбрана модель с наибольше точностью.
 
Лучшей оказалась модель "случайного леса" с точностью 0.7993779160186625. На финальной проверке этой модели с заданными гиперпараметрами на тестовой выборке точность составила  0.80248833592535.

На основании чего делаем вывод, что полученная модель рекомендована к работе.

## Проверка модели на адекватность

В качестве случайной модели используем DummyClassifier. Ее обычно применяют для сравнения с бооее сложныи моделями, например, такой, выыбрали в итоге мы - случайный лес.

In [11]:
#записываем модель в переменную
model = DummyClassifier(strategy="most_frequent")
model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
predictions_test = model.predict(features_test) # посчитаем качество модели на тестовой выборке
accuracy=accuracy_score(target_test, predictions_test)

print('Доля правильных ответов на тестовой выборке:', accuracy)

Доля правильных ответов на тестовой выборке: 0.6936236391912908


На более простой модели мы получаем точность гораздно ниде, чем на протестированных ранее. 
Accuracy модели Dummy = 0.6936236391912908, для сравнения с точностью  модели  "логистической регрессии", которая у нас на выборках дала самый низкий результат = 0.7387247278382582 (на валидационной выборке). Что явялется тестом на адекватность - наша модель работе лучше случайной.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [х] Весь код исполняется без ошибок
- [х] Ячейки с кодом расположены в порядке исполнения
- [х] Выполнено задание 1: данные загружены и изучены
- [х] Выполнено задание 2: данные разбиты на три выборки
- [х] Выполнено задание 3: проведено исследование моделей
    - [х] Рассмотрено больше одной модели
    - [х] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [х] Написаны выводы по результатам исследования
- [х] Выполнено задание 3: Проведено тестирование
- [х] Удалось достичь accuracy не меньше 0.75
